<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание №2

**

## Часть 1. ML workflow (**всего 5 баллов**)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы.

In [4]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [5]:
X = df.drop('quality', axis=1)
y = df['quality']

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [8]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Предсказания на тренировочных и тестовых данных
y_train_pred = lin_reg.predict(X_train)
y_test_pred = lin_reg.predict(X_test)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [9]:
print("MSE (train):", mean_squared_error(y_train, y_train_pred))
print("MSE (test):", mean_squared_error(y_test, y_test_pred))
print("R2 (train):", r2_score(y_train, y_train_pred))
print("R2 (test):", r2_score(y_test, y_test_pred))

MSE (train): 0.42078664921236164
MSE (test): 0.4112348717504203
R2 (train): 0.36119824413213153
R2 (test): 0.3513885332505232


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [10]:
lin_reg_cv_score = cross_val_score(lin_reg, X, y, cv=5, scoring='r2')
print("Среднее значение R2 на кросс-валидации:", np.mean(lin_reg_cv_score))

Среднее значение R2 на кросс-валидации: 0.29004162884219686


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1.

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [11]:
from sklearn.linear_model import Lasso

lasso = Lasso()
parameters = {'alpha': np.arange(0.1, 1.1, 0.1)}
lasso_grid = GridSearchCV(lasso, parameters, cv=5)
lasso_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])})

### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [12]:
print("Наилучший алгоритм:", lasso_grid.best_estimator_)
print("Наилучшее качество:", lasso_grid.best_score_)

Наилучший алгоритм: Lasso(alpha=0.1)
Наилучшее качество: 0.24002181024109745


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [13]:
best_lasso = lasso_grid.best_estimator_
y_test_pred_lasso = best_lasso.predict(X_test)
print("R2-score на тесте:", r2_score(y_test, y_test_pred_lasso))

R2-score на тесте: 0.20887882853036965


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [14]:
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('lin_reg', LinearRegression())
])

poly_r2_scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')
print("R2-score с полиномиальными признаками на кросс-валидации:", np.mean(poly_r2_scores))

R2-score с полиномиальными признаками на кросс-валидации: 0.23009617099814764


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [15]:
pipeline.fit(X_train, y_train)

# Предсказания для train и test
y_train_pred_poly = pipeline.predict(X_train)
y_test_pred_poly = pipeline.predict(X_test)

# Вывод R2-score и MSE
print("R2-score (train):", r2_score(y_train, y_train_pred_poly))
print("R2-score (test):", r2_score(y_test, y_test_pred_poly))
print("MSE (train):", mean_squared_error(y_train, y_train_pred_poly))
print("MSE (test):", mean_squared_error(y_test, y_test_pred_poly))

R2-score (train): 0.4507762892473709
R2-score (test): 0.35135570896634216
MSE (train): 0.36178047851731443
MSE (test): 0.4112556831159102


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

In [ ]:
#1) Для оценки качества исходной модели необходимо сравнить показатели R2 и MSE на тренировочных и тестовых данных. Если они сильно отличаются, это может указывать на переобучение.

#2) L1-регуляризация может помочь улучшить качество модели за счет предотвращения переобучения и отбора признаков.

#3) Добавление полиномиальных признаков может улучшить качество модели, если исходные данные имеют нелинейные зависимости. Однако это также может увеличить риск переобучения.


### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [16]:
# Пример: использование Ridge регрессии
from sklearn.linear_model import Ridge

ridge = Ridge()
parameters_ridge = {'alpha': np.logspace(-3, 3, 7)}
ridge_grid = GridSearchCV(ridge, parameters_ridge, cv=5)
ridge_grid.fit(X_train, y_train)

best_ridge = ridge_grid.best_estimator_
y_test_pred_ridge = best_ridge.predict(X_test)
print("R2-score на тесте с Ridge регрессией:", r2_score(y_test, y_test_pred_ridge))

R2-score на тесте с Ridge регрессией: 0.347282371002654


## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [57]:
sales = pd.read_csv('/content/sales_train.csv.gz.csv')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [58]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`.

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [59]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


#### Способ 2

In [60]:
'''
     Differently to `.target.mean()` function `transform`
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection.

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях.

In [61]:
from sklearn.model_selection import KFold

# Создаем объект KFold
kf = KFold(n_splits=5, shuffle=False)

# Создаем колонку для кодированных значений
all_data['item_target_enc_kfold'] = np.nan

# Проходим по каждому фолду
for train_index, val_index in kf.split(all_data):
    # Разделяем данные на тренировочные и валидационные
    X_train, X_val = all_data.iloc[train_index], all_data.iloc[val_index]

    # Вычисляем средние значения таргета по item_id на тренировочных данных
    item_id_target_mean = X_train.groupby('item_id').target.mean()

    # Заполняем кодированные значения на валидационных данных
    all_data.loc[val_index, 'item_target_enc_kfold'] = X_val['item_id'].map(item_id_target_mean)

# Заполняем NaN значения глобальным средним
all_data['item_target_enc_kfold'].fillna(0.3343, inplace=True)

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4830386988621699


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`.

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [62]:
# Суммарный таргет по всем объектам для каждого item_id
cumulative_sum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']
cumulative_count = all_data.groupby('item_id').cumcount()

# Вычисляем среднее значение таргета для каждого item_id, исключая текущий объект
all_data['item_target_enc_loo'] = cumulative_sum / cumulative_count

# Заполняем NaN значения глобальным средним
all_data['item_target_enc_loo'].fillna(0.3343, inplace=True)

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4830386988621699


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [63]:
global_mean = 0.3343

alpha = 100

# Вычисляем среднее значение target для каждой категории
category_means = all_data.groupby('item_id')['target'].mean()

# Реализуем smoothing с α=100
all_data['target_smoothed'] = (
    (category_means * all_data.groupby('item_id')['target'].count() + global_mean * alpha) /
    (all_data.groupby('item_id')['target'].count() + alpha)
)


corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4830386988621699


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [64]:
all_data.sort_values(['date_block_num', 'shop_id', 'item_id'], inplace=True)

# Вычисляем кумулятивную сумму и количество для каждой группы
all_data['cumsum'] = all_data.groupby(['shop_id', 'item_id'])['target'].cumsum() - all_data['target']
all_data['cumcount'] = all_data.groupby(['shop_id', 'item_id']).cumcount()

# Рассчитываем expanding mean, учитывая деление на 0
all_data['expanding_mean'] = all_data['cumsum'] / all_data['cumcount'].where(all_data['cumcount'] != 0, 1)

# Удаляем временные столбцы
all_data.drop(['cumsum', 'cumcount'], axis=1, inplace=True)

# Выводим корреляцию
corr = np.corrcoef(all_data['target'].values, all_data['expanding_mean'].values)[0][1]
print(corr)


corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.5814548653773877
0.4830386988621699


Ожидаемый ответ 0.5025